In [1]:
import pandas as pd

## create the dataset for gemini distillation

In [59]:
stock_mentions = "../data/stock_mentions.parquet"
stock_mentions = pd.read_parquet(stock_mentions)
stock_mentions

,ticker,company_name,original_word,match_type,match_score,text,text_index,origin,original_index
0,AAPL,Apple Inc.,AAPL,ticker,100.000000,Selling my AAPL position because of Itunes/App...,0,submission_title,0
1,LEN,Lennar,Lennar,name,100.000000,Thoughts on long-term position for Lennar Corp...,30,submission_title,30
2,AMD,Advanced Micro Devices,AMD,ticker,100.000000,Why can't I find the AMD Annual Report for 2016,40,submission_title,40
3,TSLA,"Tesla, Inc.",TSLA,ticker,100.000000,What is happening to TSLA stock?,65,submission_title,65
4,HLT,Hilton Worldwide,HLT,ticker,100.000000,My position in HLT has split into different sy...,71,submission_title,71
...,...,...,...,...,...,...,...,...,...
298602,PYPL,PayPal,PayPal,name,100.000000,This story has hit some pretty big publication...,2245688,comment,8902
298603,TSLA,"Tesla, Inc.",TSLA,ticker,100.000000,"Index fund such as FXAIX $300, TSLA $200, GOOG...",2245695,comment,8909
298604,GOOG,Alphabet Inc. (Class C),GOOG,ticker,100.000000,"Index fund such as FXAIX $300, TSLA $200, GOOG...",2245695,comment,8909
298605,NVDA,Nvidia,NVDA,ticker,100.000000,"Index fund such as FXAIX $300, TSLA $200, GOOG...",2245695,comment,8909


In [31]:
from nlp_utils import score_text_gemini

with open("api_key.txt", "r") as f:
    api_key = f.read().strip()

In [103]:
first_index = 500
last_index = 600

In [104]:
def score_row_gemini(row):
    score, tokens = score_text_gemini(row['text'], row['ticker'], api_key)
    print(f'scored {row['origin']} at index {row['original_index']}')
    return score

def score_dataframe(df, start_index=0, end_index=99):
    return df.iloc[start_index:end_index].apply(score_row_gemini, axis=1)

scores_list = score_dataframe(stock_mentions, start_index=first_index, end_index=last_index)

scored submission_title at index 5700
scored submission_title at index 5725
scored submission_title at index 5728
scored submission_title at index 5729
scored submission_title at index 5732
scored submission_title at index 5739
scored submission_title at index 5745
scored submission_title at index 5750
scored submission_title at index 5762
scored submission_title at index 5763
scored submission_title at index 5772
scored submission_title at index 5784
scored submission_title at index 5801
scored submission_title at index 5812
scored submission_title at index 5816
scored submission_title at index 5819
scored submission_title at index 5828
scored submission_title at index 5829
scored submission_title at index 5829
scored submission_title at index 5835
scored submission_title at index 5842
scored submission_title at index 5846
scored submission_title at index 5852
scored submission_title at index 5866
scored submission_title at index 5868
scored submission_title at index 5870
scored submi

In [97]:
reduced_stock_mentions = stock_mentions.iloc[first_index:last_index]
reduced_stock_mentions['score'] = scores_list

/var/folders/2n/ywnb5cc17zq6c5n2w2n7bkzw0000gn/T/ipykernel_11482/1028078542.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_stock_mentions['score'] = scores_list


In [98]:
reduced_stock_mentions.to_parquet(f"fine_tuning/data/partial/reduced_stock_mentions_{first_index}_{last_index}.parquet")

In [106]:
import os

directory = os.fsencode('fine_tuning/data/partial/')

filenames = []
    
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".parquet"): 
        filenames.append(filename)
        continue
    else:
        continue

filenames = sorted(filenames)
dataframes = []

for filename in filenames:
    dataframes.append(pd.read_parquet(f"fine_tuning/data/partial/{filename}"))

reduced_stock_mentions = pd.concat(dataframes)

In [107]:
reduced_stock_mentions

,ticker,company_name,original_word,match_type,match_score,text,text_index,origin,original_index,score
0,AAPL,Apple Inc.,AAPL,ticker,100.0,Selling my AAPL position because of Itunes/App...,0,submission_title,0,-1
1,LEN,Lennar,Lennar,name,100.0,Thoughts on long-term position for Lennar Corp...,30,submission_title,30,0
2,AMD,Advanced Micro Devices,AMD,ticker,100.0,Why can't I find the AMD Annual Report for 2016,40,submission_title,40,0
3,TSLA,"Tesla, Inc.",TSLA,ticker,100.0,What is happening to TSLA stock?,65,submission_title,65,0
4,HLT,Hilton Worldwide,HLT,ticker,100.0,My position in HLT has split into different sy...,71,submission_title,71,0
...,...,...,...,...,...,...,...,...,...,...
495,QCOM,Qualcomm,QCOM,ticker,100.0,Prediction: Massive dump of QCOM after earning...,5634,submission_title,5634,-1
496,WEC,WEC Energy Group,WEC,ticker,100.0,WEC 2nd Quarter Earnings Report Speculation,5656,submission_title,5656,0
497,TMUS,T-Mobile US,TMUS,ticker,100.0,TMUS Earnings thread,5659,submission_title,5659,0
498,MSFT,Microsoft,Microsoft,name,100.0,Microsoft about to report - buy or short?,5671,submission_title,5671,0


In [108]:
from nlp_utils import system_text

def system_row(row):
    prompt = system_text(row['ticker'])
    return prompt

reduced_stock_mentions['system_text'] = reduced_stock_mentions.apply(system_row, axis=1)
reduced_stock_mentions

,ticker,company_name,original_word,match_type,match_score,text,text_index,origin,original_index,score,system_text
0,AAPL,Apple Inc.,AAPL,ticker,100.0,Selling my AAPL position because of Itunes/App...,0,submission_title,0,-1,"You need to act as an expert linguist, trader ..."
1,LEN,Lennar,Lennar,name,100.0,Thoughts on long-term position for Lennar Corp...,30,submission_title,30,0,"You need to act as an expert linguist, trader ..."
2,AMD,Advanced Micro Devices,AMD,ticker,100.0,Why can't I find the AMD Annual Report for 2016,40,submission_title,40,0,"You need to act as an expert linguist, trader ..."
3,TSLA,"Tesla, Inc.",TSLA,ticker,100.0,What is happening to TSLA stock?,65,submission_title,65,0,"You need to act as an expert linguist, trader ..."
4,HLT,Hilton Worldwide,HLT,ticker,100.0,My position in HLT has split into different sy...,71,submission_title,71,0,"You need to act as an expert linguist, trader ..."
...,...,...,...,...,...,...,...,...,...,...,...
495,QCOM,Qualcomm,QCOM,ticker,100.0,Prediction: Massive dump of QCOM after earning...,5634,submission_title,5634,-1,"You need to act as an expert linguist, trader ..."
496,WEC,WEC Energy Group,WEC,ticker,100.0,WEC 2nd Quarter Earnings Report Speculation,5656,submission_title,5656,0,"You need to act as an expert linguist, trader ..."
497,TMUS,T-Mobile US,TMUS,ticker,100.0,TMUS Earnings thread,5659,submission_title,5659,0,"You need to act as an expert linguist, trader ..."
498,MSFT,Microsoft,Microsoft,name,100.0,Microsoft about to report - buy or short?,5671,submission_title,5671,0,"You need to act as an expert linguist, trader ..."


In [109]:
json_data = [{"messages": [{"role": "system", "content": row["system_text"]}, {"role": "user", "content": row["text"]},{"role": "assistant", "content": row["score"]}]} for _, row in reduced_stock_mentions.iterrows()]

In [110]:
json_df = pd.DataFrame(json_data)
json_df

,messages
0,"[{'role': 'system', 'content': 'You need to ac..."
1,"[{'role': 'system', 'content': 'You need to ac..."
2,"[{'role': 'system', 'content': 'You need to ac..."
3,"[{'role': 'system', 'content': 'You need to ac..."
4,"[{'role': 'system', 'content': 'You need to ac..."
...,...
494,"[{'role': 'system', 'content': 'You need to ac..."
495,"[{'role': 'system', 'content': 'You need to ac..."
496,"[{'role': 'system', 'content': 'You need to ac..."
497,"[{'role': 'system', 'content': 'You need to ac..."


In [111]:
from sklearn.model_selection import train_test_split

# Split into 80% train, 20% temp (which will be split further)
train_data, temp_data = train_test_split(json_df, test_size=0.2, random_state=42)

# Split temp into 50% validation, 50% test (so each is 10% of total)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Convert DataFrames back to list format for JSON saving
train_json = train_data.to_dict(orient="records")
val_json = val_data.to_dict(orient="records")
test_json = test_data.to_dict(orient="records")

In [112]:
import json

# Save train set
with open("fine_tuning/data/json/train.jsonl", "w", encoding="utf-8") as f:
    for item in train_json:
        f.write(json.dumps(item) + '\n')

# Save validation set
with open("fine_tuning/data/json/valid.jsonl", "w", encoding="utf-8") as f:
    for item in val_json:
        f.write(json.dumps(item) + '\n')

# Save test set
with open("fine_tuning/data/json/test.jsonl", "w", encoding="utf-8") as f:
    for item in test_json:
        f.write(json.dumps(item) + '\n')

## score with local models

In [118]:
from nlp_utils import score_text_local

model_path = 'fine_tuning/models/Llama-3.2-1B-Instruct-4bit/fused_model'

def score_row_local(row):
    score = score_text_local(row['text'], row['ticker'], model_path)
    return score

scores_series_local = reduced_stock_mentions.apply(score_row_local, axis=1)

In [123]:
model_path_raw = 'mlx-community/Llama-3.2-1B-Instruct-4bit'

def score_row_local_raw(row):
    score = score_text_local(row['text'], row['ticker'], model_path_raw)
    return score

scores_series_local_raw = reduced_stock_mentions.apply(score_row_local_raw, axis=1)

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

...

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [124]:
reduced_stock_mentions['score_local_finetuned'] = scores_series_local
reduced_stock_mentions['score_local_raw'] = scores_series_local_raw

In [125]:
reduced_stock_mentions[['ticker', 'text', 'score', 'score_local_finetuned', 'score_local_raw']]

,ticker,text,score,score_local_finetuned,score_local_raw
0,AAPL,Selling my AAPL position because of Itunes/App...,-1,1,0
1,LEN,Thoughts on long-term position for Lennar Corp...,0,-1,0
2,AMD,Why can't I find the AMD Annual Report for 2016,0,0,0
3,TSLA,What is happening to TSLA stock?,0,0,0
4,HLT,My position in HLT has split into different sy...,0,0,0
...,...,...,...,...,...
495,QCOM,Prediction: Massive dump of QCOM after earning...,-1,1,1
496,WEC,WEC 2nd Quarter Earnings Report Speculation,0,1,0
497,TMUS,TMUS Earnings thread,0,0,0
498,MSFT,Microsoft about to report - buy or short?,0,0,0


## legacy code

Here we check that the tokens used for every request are relatively low. This means that for the API we only need to worry about request rate and not token rate.

In [22]:
client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction='reply to the question and be friendly'),
    contents='how many days in a year?'
    )

response

NameError: name 'genai' is not defined

In [32]:
response.usage_metadata.total_token_count

42

legacy code:

In [ ]:
from mlx_lm import load, generate

model, tokenizer = load("mlx-community/gemma-3-1b-it-4bit")

stock = 'GME'

messages = [{
     'role': 'system',
     'content': f'''
                You need to act as an expert linguist, trader and researcher, read the reddit comment about {stock} that is given to you, and classify wether the comment is bullish or bearish about {stock}.
                Your answer should be 1 if the comment is positive about the stock, -1 if it is negative, and 0 if it is neutral. Do not reply to the comment, just classify it.
                ''',
   },
   {
     'role': 'user',
     'content': 'I think GME is going up despite the market being bearish',
   }]
prompt = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True
)

text = generate(model, tokenizer, prompt=prompt, verbose=True)

Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 96791.63it/s]


0
Prompt: 132 tokens, 355.723 tokens-per-sec
Generation: 2 tokens, 12.702 tokens-per-sec
Peak memory: 3.615 GB
